In [1]:
# Import necessary libraries
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input
from keras.layers.noise import AlphaDropout
from keras.layers import Layer
from tensorflow.python.keras import backend as K
from sklearn.model_selection import train_test_split
import pickle

# Import custom modules
from network import *
from data import *

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dbfile = open('../../Data/full', 'rb')    
data = pickle.load(dbfile)
dbfile = open('../../Data/location_pert', 'rb')    
location_pert = pickle.load(dbfile)
dbfile = open('../../Data/pert2profiles', 'rb')    
pert2profiles = pickle.load(dbfile)
dbfile = open('../../Data/test_perts', 'rb')    
test_pert = pickle.load(dbfile)
dbfile = open('../../Data/train_perts', 'rb')    
train_pert = pickle.load(dbfile)
dbfile.close()

In [3]:
X_train = generate_data(data,train_pert[0:10],42)

batch_size:  420
420/420

In [4]:
X_test = generate_data(data,test_pert[0:10],42)

batch_size:  420
420/420

# Network

In [13]:
# Create the base network
network = dict(n_dense=5, dense_units=16, activation='selu', dropout=AlphaDropout, dropout_rate=0.1,
               kernel_initializer='lecun_normal', optimizer='sgd', num_classes=2)

shared_model = create_base_network(**network)

In [14]:
# Create the siamese network
model = create_siamese_network(shared_model)
print("Siamese network model summary")
model.summary()

y_pred.shape =  Tensor("merged_layer_1/concat:0", shape=(?, 48), dtype=float32)
Siamese network model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 978)          0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 978)          0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 978)          0                                            
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 16)           16752       anchor_input[0][0]               

In [30]:
y_dum = np.zeros((len(X_train[0]),1))
model.fit([X_train[0],X_train[1],X_train[2]],y_dum,epochs=100)

Epoch 1/100
420/420 [==============================] - 1s 3ms/step - loss: 6.1394
Epoch 2/100
420/420 [==============================] - 0s 181us/step - loss: 5.3144
Epoch 3/100
420/420 [==============================] - 0s 182us/step - loss: 5.3077
Epoch 4/100
420/420 [==============================] - 0s 201us/step - loss: 4.3335
Epoch 5/100
420/420 [==============================] - 0s 205us/step - loss: 4.6981
Epoch 6/100
420/420 [==============================] - 0s 200us/step - loss: 4.0506
Epoch 7/100
420/420 [==============================] - 0s 206us/step - loss: 3.3781
Epoch 8/100
420/420 [==============================] - 0s 177us/step - loss: 3.2525
Epoch 9/100
420/420 [==============================] - 0s 190us/step - loss: 2.6984
Epoch 10/100
420/420 [==============================] - 0s 183us/step - loss: 3.0895
Epoch 11/100
420/420 [==============================] - 0s 192us/step - loss: 3.3869
Epoch 12/100
420/420 [==============================] - 0s 189us/step - loss

420/420 [==============================] - 0s 211us/step - loss: 0.0738
Epoch 97/100
420/420 [==============================] - 0s 177us/step - loss: 0.1685
Epoch 98/100
420/420 [==============================] - 0s 174us/step - loss: 0.1640
Epoch 99/100
420/420 [==============================] - 0s 174us/step - loss: 0.1201
Epoch 100/100
420/420 [==============================] - 0s 360us/step - loss: 0.0576


In [31]:
pred1 = model.predict([X_test[0],X_test[1],X_test[2]])
pred1.shape

(420, 48)

In [32]:
pred = model.predict([X_train[0],X_train[1],X_train[2]])
pred.shape

(420, 48)

In [33]:
from sklearn.preprocessing import normalize
anch = normalize(pred[0][0:16].reshape(1,-1)).flatten()
pos = normalize(pred[0][16:32].reshape(1,-1)).flatten()
neg = normalize(pred[0][32:].reshape(1,-1)).flatten()

In [34]:
def softmax_function(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [36]:
values = []
for i in pred1:
    anch = normalize(i[0:16].reshape(1,-1)).flatten()
    pos = normalize(i[16:32].reshape(1,-1)).flatten()
    neg = normalize(i[32:].reshape(1,-1)).flatten()
    pos_neg_val = softmax_function([[np.abs(np.dot(anch,pos)), np.abs(np.dot(anch,neg))]]).flatten()
    
    #print(pos_neg_val)
    values.append(pos_neg_val[0]>=0.5)
    values.append(pos_neg_val[1]<0.5)
# print(positives, negatives)
#     print(pos_val, neg_val, pos_val>neg_val)
print((np.sum(np.asarray(values)==True))/len(values))

0.5595238095238095


In [21]:
np.dot(anch,pos)

0.5174793

In [39]:
import pickle
dbfile = open('../../Data/X_train_triplet', 'ab')
pickle.dump(X_train, dbfile)
dbfile.close()

In [38]:
import pickle
dbfile = open('../../Data/X_test_triplet', 'ab')
pickle.dump(X_test, dbfile)
dbfile.close()